# Two-point statistics from *Heracles*

This notebook demonstrates how *Heracles* extracts the two-point statistics from a *LE3-WL* shear catalogue.

The statistics are done here at a much lower resolution than for the scientific goals of *Euclid*. That is because at the target resolution of *NSIDE = 4K* for the position and shear maps (1.5G per map) and *NSIDE = 8K* for the visibility and weight maps (6G per map), the memory required makes it impossible to hold all maps at once. In an actual pipeline, each map is constructed in isolation by a separate worker instance, and each worker only holds the data it requires at each step.

The example uses the [*Flagship2* data for *LE3-WL*](https://sdcuk-webdav.roe.ac.uk/LE3_WL_Flagship2/). The catalogue contains about 128 million galaxies, roughly corresponding to *Euclid* DR1 in the north.

<div class="alert alert-block alert-warning">
    <p><strong>Note:</strong> This notebook is meant to give you an idea of <strong>how <em>Heracles</em> works</strong>.</p>
    <p><ul>
        <li>The notebook does <strong>not</strong> perform a very thorough analysis.</li>
        <li>The notebook does <strong>not</strong> show everything that <em>Heracles</em> can do.</li>
        <li>The notebook does <strong>not</strong> indicate how good the eventual <em>Euclid</em> results may or may not be.</li>
        <li>The notebook does <strong>not</strong> treat all observational effects we know about.</li>
        <li>The notebook does <strong>not</strong> model the expected signal properly.</li>
    </ul></p>
    <p><strong>This is a toy, treat it is such!</strong></p>
</div>

**Contents:**

* [Setup](#Setup)
* [Basic parameters](#Basic-parameters)
* [Catalogues](#Catalogues)
* [Tomographic binning](#Tomographic-binning)
* [Visibility maps](#Visibility-maps)
* [Maps](#Maps)
* [Alms](#Alms)
* [Two-point statistics](#Two-point-statistics)
* [Noise bias](#Noise-bias)
* [Debiasing](#Debiasing)
* [Mixing matrices](#Mixing-matrices)
* [Theory](#Theory)
* [Results](#Results)

## Setup

Uncomment this to enable info-level logging. This produces quite a lot of output below, but will show exactly what is going on while you are waiting for results.

In [1]:
#import logging
#logging.basicConfig(level=logging.INFO)

Some required imports, nothing fancy.

In [2]:
import numpy as np
import healpy as hp
import fitsio
import matplotlib.pyplot as plt
import matplotlib as mpl

## Basic parameters

As initially mentioned, set this to values such that your machine does not blow up when this example keeps all maps in memory at the same time.

In [3]:
nside = 1024
lmax = 1000

## Catalogues

*Heracles* provides a flexible interface for loading catalogues from FITS files or arrays. It also provides a base that can quickly be extended e.g. to databases or more.

Here we use the FITS interface to read a catalogue from file. We could specify the columns to read, but we don't do that here, because the catalogue only has columns of interest here.

Catalogues are never read into memory all at once. The `page_size` property determines how many rows are read at a time.

In [ ]:
from heracles.catalog import FitsCatalog

In [ ]:
# load the FITS catalogue
wlfs2_cat = FitsCatalog('wlfs2_dr1n_vis24.5_nomag.fits')

Using the `.add_filter()` method, we could add filters to the catalogue here which e.g. strip rows with invalid values, or apply an extra footprint mask to the catalogue.

### Playground

The `Catalog` interface is essentially an interator over pages of catalogue data. You can use it as such:

In [ ]:
nrows = 0
for page in wlfs2_cat:
    nrows += page.size

# no need to iterate to get the number of rows, really
assert nrows == wlfs2_cat.size

print(f'there are {nrows:_} rows in your catalogue')

The `page` object is a mapping of column names to rows, with some additional features:

* The number of rows, `page.size`
* The names of columns, `page.names`
* A view of the underlying data, `page.data`
* Make a copy of the page, `page.copy()`
* Delete specific rows, `page.delete(where)`
* Return multiple columns at once, using `page['a', 'b']` or `page[['a', 'b']]`
* Return columns while checking for invalid values using `page.get('a', 'b')`

## Visibility maps

The position statistics require knowledge of the *a priori* probability of detecting a galaxy at each point in the sky. We call this the *visibility*, and we use it in the form of *visibility maps*, which are full-sky maps of detection probabilities.

In [ ]:
vmap = hp.read_map('vmap_wlfs2_dr1n_vis24.5_nomag.fits')

# fix UNSEEN pixels to zero and rescale to nside
vmap[vmap == hp.UNSEEN] = 0.
vmap = hp.ud_grade(vmap, 2*nside)

For a real survey, the visibility is a complicated function of position, observing conditions, and selection. Each tomographic bin would thus normally require an individual visibility map. However, in this simulated example, the selection is the same for all positions and tomographic bins, and the visibility map is simply the footprint map of the survey.

In [ ]:
# set visibility map of entire catalogue
wlfs2_cat.visibility = vmap

### Playground

Quick inspection of the visibility map:

In [ ]:
hp.mollview(vmap, title='visibility', cmap='binary', bgcolor='none')
hp.graticule()
plt.show()

## Tomographic binning

The `wlfs2_cat` object can be used to read the entire catalogue. However, we would like to split our galaxies into individual tomographic bins. In fact, each galaxy has already been assigned a label that says which tomographic bin it belongs to, in the `TOM_BIN_ID` column.

To perform the tomographic binning, we construct a `dict` that assigns a bin ID of type `int` to a catalogue. We perform a selection on the `wlfs2_cat` FITS using the `[]` syntax, which returns a new view of the `FitsCatalog` with the given rowfilter applied.

In [ ]:
catalogs = {i: wlfs2_cat[f'TOM_BIN_ID == {i}'] for i in range(10)}

In [ ]:
catalogs

View have their own invididual `.visibility` property, but inherit the visibility of the base catalogue by default.

Instead of the `[]` syntax, view can also be created using the `.where()` method, in which case the visibility can be given directly.

In [ ]:
catalogs[3].visibility is vmap

Note that `catalogs` is a simple mapping of integer bin IDs to instances of type `Catalog`. You can create such a mapping in any way you like; the values do not have to come from the same `FitsCatalog`, or even the same type of catalogue.

### Playground

Return the number of rows in tomographic bin 5.

In [ ]:
# need to sum because FITS cannot tell the size a priori
nrows = sum(page.size for page in catalogs[5])

print(f'tomographic bin 5 contains {nrows:_} rows')

## Maps

In [ ]:
from heracles.maps import PositionMap, ShearMap, map_catalogs

The next step is to produce position and shear maps from the catalogues for each tomographic bin. *Heracles* can construct maps from individual catalogues, and it can map a whole set of catalogues all at once. Here we use the latter interface, which is available as the `map_catalogs()` functions.

To specify which maps to make, we construct a mapping from names to objects that do the map-making. Each of these receives a list of columns that it reads, plus potentially some other options.

For a standard 3x2pt analysis in harmonic space, we need

* A position map (P) for angular clustering and galaxy-galaxy lensing;
* A shear map (G) for cosmic shear and galaxy-galaxy lensing.

We set that the shear maps should flip the sign of the "G2" column (`conjugate=True`).

In [ ]:
# don't repeat the lon and lat column every time
lonlat = ('RIGHT_ASCENSION', 'DECLINATION')

maps = {
    'P': PositionMap(nside, *lonlat),
    'G': ShearMap(nside, *lonlat, 'G1', 'G2', 'WEIGHT', conjugate=True),
}

Now we are able to create maps of the tomographic catalogues. We only need to pass in the catalogue and mappings constructed above.

In [ ]:
data = map_catalogs(maps, catalogs, parallel=True, progress=True)

The resulting `data` mapping has keys consisting of a map ID (`P`, `G`) and a bin ID. Results from *Heracles* are always of this form, which we call "toc dicts", short for "table of contents dictionaries" from how the data was originally written to disk.

In [ ]:
list(data.keys())[:8] + ['...']

### Playground

We can take a quick look at the maps for a tomographic bin ID, say 3.

In [ ]:
i = 3

fig, ax = plt.subplots(2, 2, figsize=(12, 8))
fig.tight_layout()
plt.sca(ax[0, 0])
hp.cartview(data['P', i], title='P', cmap='binary', min=-1., max=4., hold=True,
            lonra=[60, 150], latra=[20, 65])
ax[0, 1].axis('off')
plt.sca(ax[1, 0])
hp.cartview(data['G', i][0], title='G1', cmap='RdGy', min=-4, max=4, hold=True,
            lonra=[60, 150], latra=[20, 65])
plt.sca(ax[1, 1])
hp.cartview(data['G', i][1], title='G2', cmap='RdGy', min=-4, max=4, hold=True,
            lonra=[60, 150], latra=[20, 65])
plt.show()

## Alms

Since we are working in harmonic space, the real-space maps we created so far are not what we are actually after. We therefore transform them into harmonic-space maps (i.e. $a_{lm}$) using the `transform_maps()` function.

In [ ]:
from heracles.maps import transform_maps

In [ ]:
alms = transform_maps(data, lmax=lmax, use_pixel_weights=True, progress=True)

The resulting `alm` object is another toc dict with keys for map ID (`P`, `G_E`, `G_B`) and bin ID.

In [ ]:
list(alms.keys())[:9] + ['...']

## Two-point statistics

We are now able to compute the two-point statistics in the form of angular power spectra. We simply call the `angular_power_spectrum()` function on the `alms`.

In [ ]:
from heracles.twopoint import angular_power_spectra

In [ ]:
cls = angular_power_spectra(alms, lmax=lmax)

We could limit the `cls` we want computed using the `include=` and `exclude=` parameters to `angular_power_specta()`, but here we have computed all combinations. They are very many, arranged in a toc dict with entries such as `('P', 'G_E', 5, 4)` for the position and *E*-mode angular power spectrum between bin IDs 5 and 4 (in order).

In [ ]:
list(cls.keys())[:6] + ['...'] + list(cls.keys())[-6:]

## Noise bias

Before we can take a meaningful look at the result, we must consider a number of observational effects that influence the angular power spectra. First is the so-called noise bias, which is owed to the fact that we are observing discrete positions and shears instead of continuous fields.

In the near future, *Heracles* will provide theoretical values for all noise biases. For the time being, we can use the `random_noisebias()` function to estimate the noise bias from randomised position and shear maps instead.

In [ ]:
from heracles.twopoint import random_noisebias

# fix seed to always get the same result in this example (don't do that in production!)
np.random.seed(12345)

In [ ]:
nbs = random_noisebias(maps, catalogs, parallel=True, progress=True,
                       lmax=lmax, use_pixel_weights=True)

## Debiasing

With the noise biases computed, we can debias our angular power spectra by subtracting them. Since we do not need the biased `cls`, we compute the debiased ones in place.

In [ ]:
from heracles.twopoint import debias_cls

In [ ]:
debias_cls(cls, noisebias=nbs, inplace=True);

## Mixing matrices

The second observational effect is that we do not have data over the entire sky, but are limited by the survey footprint and visibility. This effect is modelled at the level of two-point statistics by the so-called mixing matrices.

Computing the mixing matrices requires separate maps for the visibility and shear weights in each tomographic bin. The mixing matrices for angular power spectra up to `lmax` require `2*lmax` angular modes themselves, so we double the resolution.

In [ ]:
nside_mm = 2*nside
lmax_mm = 2*lmax

We now use the same catalogues as before to create the visibility and weight maps, using the `map_catalogs()` function as before. Of course, we could have also computed all maps in one go.

In [ ]:
from heracles.maps import VisibilityMap, WeightMap

In [ ]:
# visibility maps are taken as-is from catalogue, so no columns
maps_mm = {
    'V': VisibilityMap(nside_mm),
    'W': WeightMap(nside_mm, *lonlat, 'WEIGHT'),
}

data_mm = map_catalogs(maps_mm, catalogs, parallel=True, progress=True)

The output has the same format as earlier, but now contains `V` and `W` maps as expected.

In [ ]:
list(data_mm.keys())[:15]

Next we transform the maps for the mixing matrices ...

In [ ]:
alms_mm = transform_maps(data_mm, progress=True,
                         lmax=lmax_mm, use_pixel_weights=True)

... and compute their angular power spectra.

In [ ]:
cls_mm = angular_power_spectra(alms_mm, lmax=lmax_mm)

To compute the mixing matrices correctly, we must divide the kernels of the HEALPix discretisation (i.e. the pixel window function) from the measured angular power spectra, to obtain what would have been measured for a continuous field.  We can do so in place.

In [ ]:
from heracles.twopoint import depixelate_cls

In [ ]:
depixelate_cls(cls_mm, inplace=True);

With the angular power spectra of visibility and weight maps available, we can compute all mixing matrices with the `mixing_matrices()` function.

In [ ]:
from heracles.twopoint import mixing_matrices

In [ ]:
mms = mixing_matrices(cls_mm, l1max=lmax, l2max=lmax, l3max=lmax_mm)

The mixing matrices are returned in a toc dict with slightly different names: They use a two letter code that indicates which angular power spectra are mixed by a given matrix:

* `00` for `(P, P) → (P, P)`;
* `0+` for `(P, E) → (P, E)` and `(P, B) → (P, B)`;
* `++` for `(E, E) → (E, E)` and `(B, B) → (B, B)`;
* `--` for `(E, E) → (B, B)` and `(B, B) → (E, E)`;
* `+-` for `(E, B) → (E, B)`.

For more details, see the paper by Brown, Castro & Taylor (2005). Note that the `+-` mixing matrix here is not the $W^{+-}$ matrix from said paper.

In [ ]:
list(mms.keys())[:15]

Finally, we add the effect of the HEALPix discretisation to the mixing matrices, so that the product of mixing matrices and expected angular power spectra models our observation.

In [ ]:
from heracles.twopoint import pixelate_mms_healpix

In [ ]:
pixelate_mms_healpix(mms, nside, inplace=True);

### Playground

Have a look at the visibility and weight map for a bin ID, say 7.

In [ ]:
i = 7

fig, ax = plt.subplots(1, 2, figsize=(12, 8))
fig.tight_layout()
plt.sca(ax[0])
hp.cartview(data_mm['V', i], title='V', cmap='binary', hold=True,
            lonra=[60, 150], latra=[20, 65])
plt.sca(ax[1])
hp.cartview(data_mm['W', i], title='W', cmap='binary', min=0, max=100, hold=True,
            lonra=[60, 150], latra=[20, 65])
plt.show()

Let's also look at one of the resulting mixing matrices, say the `++` one.

In [ ]:
plt.imshow(mms['++', i, i], cmap='binary',
            norm=mpl.colors.LogNorm(vmin=1e-4))
plt.colorbar(pad=0.025, fraction=0.0465)
plt.show()

## Theory

The final piece of the puzzle is a set of theory angular power spectra that model the full sky signal. Here we use *CAMB* to get a quick and dirty approximation of what the proper theory spectra might look like.

In [ ]:
import camb
from camb.sources import SplinedSourceWindow

We set the *CAMB* cosmology to match the *Flagship2* simulation, which is

* $\Omega_m = 0.319$,
* $\Omega_b = 0.049$,
* $\Omega_\Lambda + \Omega_\gamma = 0.681$,
* $A_s = 2.1 \cdot 10^{-9}$,
* $n_s = 0.96$,
* $h = 0.67$.

Also set some other properties that come straight from the *CAMBdemo* notebook.

In [ ]:
pars = camb.CAMBparams()
pars.set_cosmology(H0=67., omch2=0.319*0.67**2, ombh2=0.049*0.67**2)
pars.InitPower.set_params(As=2.1e-9, ns=0.96)
pars.Want_CMB = False 
pars.NonLinear = camb.model.NonLinear_both
pars.set_for_lmax(lmax, lens_potential_accuracy=1)

We need the redshift distributions for the tomographic bins. For the real data, these will be provided by *OU-PHZ* in some *TBD* format. Here, we use the file from the *Flagship2* catalogue for *LE3-WL* without asking any further questions.

In [ ]:
nz = fitsio.read('nz_wlfs2_dr1n_vis24.5_nomag.fits')

To compute the theory spectra of the galaxy number counts, we need the bias factor between matter and galaxies. This is a bias factor fitted to *Flagship2*, kindly provided by Isaac Tutusaus.

In [ ]:
z = nz['Z_MID']
bz = 0.83070341 + 1.19054721*z - 0.92835749*z**2 + 0.42329232*z**3

Given redshift distributions and bias factor, we can construct the *CAMB* source distributions for positions (counts) and shears (lensing).

In [ ]:
sources = []
for i in range(len(catalogs)):
    nz_i = nz[f'BIN{i}']
    sources += [
        SplinedSourceWindow(source_type='counts', z=z, W=nz_i, bias_z=bz),
        SplinedSourceWindow(source_type='lensing', z=z, W=nz_i),
    ]
pars.SourceWindows = sources

Use the `pars` we constructed above to compute the full sky theory spectra up to `lmax`, setting `raw_cl=True` to return unscaled $C_l$ values.

In [ ]:
results = camb.get_results(pars)
camb_cls = results.get_source_cls_dict(lmax=lmax, raw_cl=True)

This is the factor that's needed to convert from convergence to shear $C_l$.

In [ ]:
l = np.arange(lmax+1)
fl = -np.sqrt((l+2)*(l+1)*l*(l-1))
fl /= np.clip(l*(l+1), 1, None)

Now we can compute the theory spectra for our observations, using the *CAMB* results and the mixing matrices we computed earlier. We store everything in a toc dict using the same format as before.

In [ ]:
theory_cls = {}
for i, s1 in enumerate(sources):
    for j, s2 in enumerate(sources[i:]):
        i1, i2 = i//2, (i+j)//2

        cl = camb_cls[f'W{i+1}xW{i+j+1}']

        if s1.source_type == 'counts' and s2.source_type == 'counts':
            theory_cls['P', 'P', i1, i2] = mms['00', i1, i2] @ cl
        elif s1.source_type == 'lensing' and s2.source_type == 'lensing':
            theory_cls['G_E', 'G_E', i1, i2] = mms['++', i1, i2] @ (cl*fl**2)
            theory_cls['G_B', 'G_B', i1, i2] = mms['--', i1, i2] @ (cl*fl**2)
        elif s1.source_type == 'counts' and s2.source_type == 'lensing':
            theory_cls['P', 'G_E', i1, i2] = mms['0+', i1, i2] @ (cl*fl)
            theory_cls['P', 'G_B', i1, i2] = np.zeros_like(cl)
        elif s1.source_type == 'lensing' and s2.source_type == 'counts':
            theory_cls['P', 'G_E', i2, i1] = mms['0+', i2, i1] @ (cl*fl)
            theory_cls['P', 'G_B', i2, i1] = np.zeros_like(cl)

## Results

Without much further ado, let's plot the results for positions and shears.  Fortunately, *Heracles* has plot routines which make this very easy.

In [ ]:
from heracles.plot import postage_stamps
from heracles.util import toc_filter
from cycler import cycler

In [ ]:
nat = np.sqrt((2*l+1)/(4*np.pi))

In [ ]:
postage_stamps([toc_filter(cls, [("G_E", "G_E"), ("G_B", "G_B")]),
                toc_filter(theory_cls, [("G_E", "G_E"), ("G_B", "G_B")])],
               [toc_filter(cls, [("P", "P")]),
                toc_filter(theory_cls, [("P", "P")])],
               shift_transpose=2, hatch_empty=True, scale=nat)
plt.show()

In [ ]:
postage_stamps([toc_filter(cls, [("P", "G_E"), ("P", "G_B")]),
                toc_filter(theory_cls, [("P", "G_E"), ("P", "G_B")])],
               scale=nat)
plt.show()

While agreement is not perfect, it's good enough, given how haphazardly we have performed the analysis and modelling of the expectation. We can certainly do better: For example, the missing power in the clustering signal at the lower end is likely due to the finite box size of the simulation, and associated lack of large-scale density perturbations (hat tip to Alex Hall for pointing this out). And there are intrinsic alignments in the simulation, but not in the theory. All of this and more must be modelled properly in a serious analysis.